In [13]:
import pandas as pd
from df_functions import read_csv_file, extract_users, format_data, split_and_reformat, clean_reqlogs, calculate_variance, count_unique_reqs, longestConsecutive

In [123]:
data = read_csv_file('requests.csv')
data

,timestamp,userID,sessionID,expiring,URL
0,1676373343936,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/442
1,1676373344344,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/407
2,1676373344456,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/969
3,1676373344912,user-65,fd48a9f9-67f4-4185-9bcc-a76b7139401c,1676373604676,/searchUsers?page=0
4,1676373345020,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/148
...,...,...,...,...,...
263206,1676444990852,user-629,2a43fe7f-d300-470d-9523-d362b70b37d3,1676445248826,/profile/572
263207,1676444991104,user-629,2a43fe7f-d300-470d-9523-d362b70b37d3,1676445248826,/conversation/572
263208,1676444991342,user-629,2a43fe7f-d300-470d-9523-d362b70b37d3,1676445248826,/conversation/572/markAsRead
263209,1676444991375,user-1005,0e84ab80-bfa4-402b-9ae0-6397441e91d6,1676445040536,/profile/995


In [57]:
"""
#code used only for retrieving full userID list to identify userId for spammers/scraper
unique_users = df.users.unique()
for element in unique_users:
    print(element)
"""

'\n#code used only for retrieving full userID list to identify userId for spammers/scraper\nunique_users = df.users.unique()\nfor element in unique_users:\n    print(element)\n'

In [124]:
#group all column values based on userID
data = data.groupby('userID').agg(lambda x: ', '.join(x.astype(str))).reset_index()


In [151]:
#dropping 'expiring' for now..
data_drop_expire = data.drop(columns='expiring')
data_drop_expire

,userID,timestamp,sessionID,URL
0,user-10,"1676375523348, 1676375529188, 1676375538350, 1...","null, 53f2f0c8-719b-4df9-837a-134f93c27501, ...","/login, /searchUsers?page=0, /profile/372, ..."
1,user-100,"1676376789336, 1676376798053, 1676376799611, 1...","null, c8e78456-e1d4-4c6e-b557-a6967d89ef02, ...","/login, /searchUsers?page=0, /profile/716, ..."
2,user-1000,"1676385815691, 1676385823348, 1676385827565, 1...","null, c48e67f9-4028-406e-b79b-1a9b37286967, ...","/login, /searchUsers?page=0, /profile/626, ..."
3,user-1001,"1676373842170, 1676373844374, 1676373849642, 1...","null, d231871c-dbe3-4069-baab-2d5f1d3ce280, ...","/login, /searchUsers?page=0, /profile/117, ..."
4,user-1002,"1676373938506, 1676373945276, 1676373946891, 1...","null, 7973a2b2-9a41-4ab2-8e69-c91c6ab20085, ...","/login, /searchUsers?page=0, /profile/162, ..."
...,...,...,...,...
999,user-995,"1676375493354, 1676375502292, 1676375509215, 1...","null, 368f244a-7878-4192-ac83-9ac8896d9736, ...","/login, /searchUsers?page=0, /profile/337, ..."
1000,user-996,"1676375215327, 1676375224782, 1676375232642, 1...","null, 5bdc4e86-69d0-43c4-b6d3-a7347dc65847, ...","/login, /searchUsers?page=0, /profile/642, ..."
1001,user-997,"1676377442299, 1676377444743, 1676377453055, 1...","null, a77fe519-7956-4ca5-b0ed-2e033fdb15e4, ...","/login, /searchUsers?page=0, /profile/760, ..."
1002,user-998,"1676377432947, 1676377439469, 1676377444691, 1...","null, da839b7f-6596-40cc-9cb9-195db960f2f6, ...","/login, /searchUsers?page=0, /profile/944, ..."


In [152]:
data_drop_expire.dtypes

userID       object
timestamp    object
sessionID    object
URL          object
dtype: object

In [ ]:
#trying to count unique sessionID... does not work!!

num_categories = data_drop_expire.groupby('userID')['sessionID'].nunique()
data_drop_expire['unique_Sessions'] = data_drop_expire['userID'].map(num_categories)
data_drop_expire

**Code below is just for checking some things**
Timestamp on other format?
Sort sequences (5min?)

In [72]:
#all requests by user-1003 (spammer 1, index = [5] in og dataframe)
spammer_1_data = data.loc[data['userID'].str.strip() == 'user-1003']
spammer_1_data

,timestamp,userID,sessionID,expiring,URL
0,1676373343936,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/442
2,1676373344456,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/969
26,1676373348109,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/searchUsers?page=2036
29,1676373348626,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/647
31,1676373349146,user-1003,c85cbd00-d2fc-4cda-9f5c-66616847d851,1676373495816,/chat/send/886
...,...,...,...,...,...
263167,1676444978796,user-1003,38b28ccc-4046-4dbe-bc7b-794319da0a73,1676445198086,/chat/send/934
263194,1676444987435,user-1003,38b28ccc-4046-4dbe-bc7b-794319da0a73,1676445198086,/searchUsers?page=11735
263196,1676444987946,user-1003,38b28ccc-4046-4dbe-bc7b-794319da0a73,1676445198086,/chat/send/474
263202,1676444990327,user-1003,38b28ccc-4046-4dbe-bc7b-794319da0a73,1676445198086,/searchUsers?page=11736


In [ ]:
#spammer_1_data.sessionID.unique()


In [157]:
#number of unique 'sessionID'
spammer_1_data['sessionID'].nunique()

238

In [52]:
#all requests by user-1004 (spammer 2, index = [6] in og dataframe)
spammer_2_data = data.loc[data['userID'].str.strip() == 'user-1004']
spammer_2_data

,timestamp,userID,sessionID,expiring,URL
39,1676373350815,user-1004,c0fb34fe-682f-4509-958b-b0c003119c49,1676373505796,/searchUsers?page=1981
50,1676373353057,user-1004,c0fb34fe-682f-4509-958b-b0c003119c49,1676373505796,/searchUsers?page=1982
55,1676373353576,user-1004,c0fb34fe-682f-4509-958b-b0c003119c49,1676373505796,/chat/send/474
89,1676373361701,user-1004,c0fb34fe-682f-4509-958b-b0c003119c49,1676373505796,/searchUsers?page=1983
93,1676373362216,user-1004,c0fb34fe-682f-4509-958b-b0c003119c49,1676373505796,/chat/send/651
...,...,...,...,...,...
263179,1676444983356,user-1004,c8a7b295-7248-4667-a6a6-2a7b3b10722a,1676445257086,/chat/send/385
263193,1676444987391,user-1004,c8a7b295-7248-4667-a6a6-2a7b3b10722a,1676445257086,/searchUsers?page=11702
263195,1676444987906,user-1004,c8a7b295-7248-4667-a6a6-2a7b3b10722a,1676445257086,/chat/send/122
263198,1676444988426,user-1004,c8a7b295-7248-4667-a6a6-2a7b3b10722a,1676445257086,/chat/send/532


In [88]:
#spammer_2_data.sessionID.unique()

In [87]:
spammer_2_data['sessionID'].nunique()

238

In [54]:
#all requests by user-1005 (data scraper, index = [7] in og dataframe)
data_scraper = data.loc[data['userID'].str.strip() == 'user-1005']
data_scraper

,timestamp,userID,sessionID,expiring,URL
1,1676373344344,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/407
4,1676373345020,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/148
6,1676373345277,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/223
7,1676373345452,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/916
8,1676373345844,user-1005,a0038f81-b46a-4f26-a7ea-2791d53404fb,1676373476316,/profile/337
...,...,...,...,...,...
263191,1676444986859,user-1005,0e84ab80-bfa4-402b-9ae0-6397441e91d6,1676445040536,/searchUsers?page=8221
263192,1676444987334,user-1005,0e84ab80-bfa4-402b-9ae0-6397441e91d6,1676445040536,/profile/729
263204,1676444990599,user-1005,0e84ab80-bfa4-402b-9ae0-6397441e91d6,1676445040536,/searchUsers?page=8222
263209,1676444991375,user-1005,0e84ab80-bfa4-402b-9ae0-6397441e91d6,1676445040536,/profile/995


In [91]:
data_scraper['sessionID'].nunique()

238

In [93]:
normal_user = data.loc[data['userID'].str.strip() == 'user-975']
normal_user

,timestamp,userID,sessionID,expiring,URL
38348,1676383289798,user-975,null,0,/login
38358,1676383294111,user-975,c12cbaa5-a55b-4a94-ba7c-495110307f12,1676383589806,/searchUsers?page=0
38379,1676383300037,user-975,c12cbaa5-a55b-4a94-ba7c-495110307f12,1676383589806,/profile/369
38380,1676383300056,user-975,c12cbaa5-a55b-4a94-ba7c-495110307f12,1676383589806,/photos/369/like
38389,1676383301625,user-975,c12cbaa5-a55b-4a94-ba7c-495110307f12,1676383589806,/config
...,...,...,...,...,...
234181,1676436890765,user-975,ae10ee26-8a5b-4255-9d46-2b9e4e5c1349,1676437127456,/profile/508
234182,1676436891130,user-975,ae10ee26-8a5b-4255-9d46-2b9e4e5c1349,1676437127456,/conversation/508
234191,1676436891484,user-975,ae10ee26-8a5b-4255-9d46-2b9e4e5c1349,1676437127456,/conversation/508/markAsRead
234192,1676436891496,user-975,ae10ee26-8a5b-4255-9d46-2b9e4e5c1349,1676437127456,/chat/send/508


In [94]:
normal_user['sessionID'].nunique()

8